# Скрипт парсинга данных с различных источников, записи их в БД и записи предсказаний в БД

Подключение необходимых библиотек

In [1]:
import requests
import json
import csv
from influxdb import InfluxDBClient

Настройки подключения к БД

In [5]:
clientdb = InfluxDBClient(host='localhost',port='47086')
clientdb.switch_database('TEST')

URL = 'http://br.so-ups.ru/webapi/api/CommonInfo/GenConsum?oesTerritory[]=530000'
start_date = '2018.11.28'
end_date = '2020.11.25'
#https://br.so-ups.ru/webapi/api/CommonInfo/GenConsum?priceZone[]=1&priceZone[]=2&oesTerritory[]=530000&startDate=2020.11.28&endDate=2020.11.28

Функции для получения данных об энергопотреблении с сайта br.so-ups.ru

In [6]:
def get_data(url, start, end):
    payload = {'startDate':start, 'endDate':end}
    r= requests.get(url,params=payload )
    data = r.json()
    return data

Функция для фильтрации входных данных и записи их в БД

In [7]:
def get_clear_data(data):
    data_list = data['m_Item2']
    generation = list()
    consuming = list()
    with open('predictions.txt', mode='r', newline='') as file:
        network_list = str(file.read().replace('\\r\\n','')).split(',')
        network_list = network_list + network_list + network_list[0:3000]
        network_list = network_list[0:len(data_list)]
        print(len(data_list),len(network_list))
    for i in range(len(data_list)):
        data_list[i]['PREDICTION'] = float(network_list[i])

    for item in data_list:
        generation.append(item['E_USE_FACT']/60000)
        consuming.append(item['GEN_FACT']/60000)
        clientdb.write_points([{"measurement": "test","tags": {"type": "generation"},"time": item['M_DATE'],"fields": {"value": generation[-1]*60000}},{"measurement": "test","tags": {"type": "consuming"},"time": item['M_DATE'],"fields": {"value": consuming[-1]*60000}},{"measurement": "test","tags": {"type": "prediction"},"time": item['M_DATE'],"fields": {"value": item['PREDICTION']}},{"measurement": "error","tags": {"error": "error"},"time": item['M_DATE'],"fields": {"value": (consuming[-1] - generation[-1])*60000}},{"measurement": "error","tags": {"sum": "sum"},"time": item['M_DATE'],"fields": {"value": (consuming[-1] - generation[-1])*60000*5500}}])
    return (generation, consuming)

Функция для чтения погоды из файла WEATHER.csv

In [8]:
def read_csv():
    temp_list = list()
    with open('WEATHER.csv', newline='', encoding='UTF-8') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            hour = int(row[0][11:13])
            day = int(row[0][0:2]) 
            mounth = int(row[0][3:5])
            for i in range(3):
                hour_list = list()
                if i == 0:
                    hour_list.append((float(row[1])))
                    hour_list.append(hour)
                    hour_list.append(day)
                    hour_list.append(mounth)
                    temp_list.append(hour_list)
                else:
                    if hour == 0:
                        hour = 24
                    hour = hour - 1
                    hour_list.append((float(row[1])))
                    hour_list.append(hour)
                    hour_list.append(day)
                    hour_list.append(mounth)
                    temp_list.append(hour_list) 
    temp_list.reverse()
    return temp_list

Вызываем все функции

In [9]:
data = get_data(URL, start_date,end_date)
data = data[0]

get_clear_data(data)[0][228:17496]
read_csv()

17496 17496


[[8.0, 0, 28, 11],
 [8.0, 1, 28, 11],
 [8.0, 2, 28, 11],
 [7.6, 3, 28, 11],
 [7.6, 4, 28, 11],
 [7.6, 5, 28, 11],
 [8.0, 6, 28, 11],
 [8.0, 7, 28, 11],
 [8.0, 8, 28, 11],
 [10.0, 9, 28, 11],
 [10.0, 10, 28, 11],
 [10.0, 11, 28, 11],
 [9.5, 12, 28, 11],
 [9.5, 13, 28, 11],
 [9.5, 14, 28, 11],
 [10.2, 15, 28, 11],
 [10.2, 16, 28, 11],
 [10.2, 17, 28, 11],
 [6.0, 18, 28, 11],
 [6.0, 19, 28, 11],
 [6.0, 20, 28, 11],
 [4.0, 21, 28, 11],
 [4.0, 22, 28, 11],
 [4.0, 23, 28, 11],
 [4.0, 0, 29, 11],
 [4.0, 1, 29, 11],
 [4.0, 2, 29, 11],
 [3.2, 3, 29, 11],
 [3.2, 4, 29, 11],
 [3.2, 5, 29, 11],
 [2.8, 6, 29, 11],
 [2.8, 7, 29, 11],
 [2.8, 8, 29, 11],
 [10.2, 9, 29, 11],
 [10.2, 10, 29, 11],
 [10.2, 11, 29, 11],
 [12.8, 12, 29, 11],
 [12.8, 13, 29, 11],
 [12.8, 14, 29, 11],
 [11.0, 15, 29, 11],
 [11.0, 16, 29, 11],
 [11.0, 17, 29, 11],
 [7.0, 18, 29, 11],
 [7.0, 19, 29, 11],
 [7.0, 20, 29, 11],
 [5.0, 21, 29, 11],
 [5.0, 22, 29, 11],
 [5.0, 23, 29, 11],
 [3.6, 0, 30, 11],
 [3.6, 1, 30, 11],
 [3.6, 